In [1]:
import pandas as pd
path_name = 'https://github.com/reckoning-machines/datasets/blob/main/betas_table_zip.zip?raw=true'
df = pd.read_csv(path_name,compression = 'zip')
featureset_columns = ['hyg_return', 'tlt_return', 'vb_return',
        'vtv_return', 'vug_return', 'rut_return', 'spx_return', 'DGS10_return',
        'DGS2_return', 'DTB3_return', 'DFF_return', 'T10Y2Y_return',
        'T5YIE_return', 'BAMLH0A0HYM2_return', 'DEXUSEU_return', 'KCFSI_return',
        'DRTSCILM_return', 'RSXFS_return', 'MARTSMPCSM44000USS_return',
        'H8B1058NCBCMG_return', 'DCOILWTICO_return', 'VXVCLS_return',
        'H8B1247NCBCMG_return', 'SP500_return', 'GASREGW_return',
        'CSUSHPINSA_return', 'UNEMPLOY_return']
df[featureset_columns]

,hyg_return,tlt_return,vb_return,vtv_return,vug_return,rut_return,spx_return,DGS10_return,DGS2_return,DTB3_return,...,RSXFS_return,MARTSMPCSM44000USS_return,H8B1058NCBCMG_return,DCOILWTICO_return,VXVCLS_return,H8B1247NCBCMG_return,SP500_return,GASREGW_return,CSUSHPINSA_return,UNEMPLOY_return
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
1,0.008968,0.000477,0.021762,0.017195,0.010297,0.020654,0.014144,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
2,-0.001524,-0.000238,-0.012042,-0.005151,-0.017760,-0.014951,-0.010863,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
3,0.005087,0.001428,-0.001410,0.004248,0.005342,0.000519,0.005529,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
4,0.003290,0.008476,0.031877,0.026305,0.021868,0.032005,0.024760,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25183,0.000518,-0.002954,0.001330,-0.004277,0.005276,0.003235,0.000916,0.004525,-0.002070,0.000000,...,0.0,0.0,0.0,-0.003306,0.008578,0.0,0.000916,-0.006652,0.0,0.0
25184,-0.000776,0.005158,-0.002435,0.000675,0.003858,-0.001969,0.002502,-0.006757,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,-0.011339,0.0,0.002502,0.000000,0.0,0.0
25185,-0.002200,0.001201,-0.006702,-0.003864,-0.002261,-0.007935,-0.002706,0.004535,0.008299,0.000000,...,0.0,0.0,0.0,0.000000,0.025806,0.0,-0.002706,0.000000,0.0,0.0
25186,-0.003502,-0.006434,-0.015192,-0.012621,-0.004136,-0.015996,-0.007381,0.009029,0.010288,0.001905,...,0.0,0.0,0.0,0.000000,0.025157,0.0,-0.007381,0.000000,0.0,0.0


In [2]:
import modelbit
mb = modelbit.login()


In [3]:
import multiprocessing
import pandas as pd
from copy import deepcopy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

class WalkForward:
    """
    A class representing a recordset walkforward back test.

    Attributes:
        test_size (float): size in % test in train / test split.
        target_column (int): single classifier target column in featureset.
        featureset_columns (list): list of featureset columns to be converted to float.
        classifier (scikit classifier): classifier object.
        minimum_dataset_length (float): length in % of initial recordset from which to begin walk forward
    """

    def __init__(self, classifier = None, dataset = None, target_column='',featureset_columns = []):
        """
        Initializes WalkForward object
        """
        self.dataset = dataset
        self.test_size = .2
        self.target_column = target_column
        self.featureset_columns = featureset_columns
        self.target_column=target_column
        self.classifier = classifier
        self.minimum_dataset_length = .2
        return

    def fit(self):
        """
        Fit the classifier
        Initially set the minimum dataset length, then step over each record
        instantiating a WalkForwardSingleton for each record
        then, appending a rolling_df_list with the output of WalkForwardSingleton (a greedy approach for debug)
        concatenate the rolling df list into a result dataset property
        """
        len_df = int(len(self.dataset))
        len_end_x = int(round(len_df * self.minimum_dataset_length,0))
        incremental_end_x = len_end_x
        rolling_df_list = []
        singleton_list = []
        for i in range(len_df - len_end_x):
            incremental_window_data = self.dataset.iloc[0:incremental_end_x]
            clsWalkForwardSingleton = WalkForwardSingleton(classifier = self.classifier, dataset = incremental_window_data, target_column=self.target_column, featureset_columns=self.featureset_columns)
            incremental_end_x = incremental_end_x + 1
            rolling_df_list.append(clsWalkForwardSingleton.singleton_dataset)
            singleton_list.append(deepcopy(WalkForwardSingleton))
        self.result_dataset = pd.concat(rolling_df_list)
        return

class WalkForwardSingleton(multiprocessing.Process):
    """
    A class representing a singleton of a recordset walkforward back test.
    Inherits from multi processing Process for threading

    Attributes:
        test_size (float): size in % test in train / test split.
        target_column (int): single classifier target column in featureset.
        featureset_columns (list): list of featureset columns to be converted to float.
        classifier (scikit classifier): classifier object.
        dataset (pandas dataframe): pandas dataframe for splitting and fitting
    """

    def __init__(self, classifier = None, dataset = None, target_column = '', featureset_columns=[], test_size=.2):
        """
        Initializes WalkforwardSingleton object
        split the dataset, fit the classifier,
        compute predictions, accuracy score and probabilities (assumes the classifer has this method)
        stores weights in a property
        converts 'dataset' to 'singleton_dataset' which appends the predictions
        """
        self.dataset = dataset
        X_train, X_test, y_train, y_test = train_test_split(dataset[featureset_columns], dataset[target_column], test_size = test_size, random_state = 10)
        classifier.fit(X_train, y_train)
        self.y_pred = list(classifier.predict(X_test))
        self.accuracy_score = accuracy_score(y_test, self.y_pred)
        self.y_pred_proba = list(classifier.predict_proba(X_test)[:,1])
        self.weights = classifier.coef_
        self.weights = pd.DataFrame(zip(X_train.columns, np.transpose(self.weights[0])), columns=['features', 'coef']) 
        self.singleton_dataset = self.dataset.tail(1)
        self.y_pred = self.y_pred[-1]
        self.y_pred_proba = self.y_pred_proba[-1]
        self.classifier = classifier
        self.singleton_dataset['y_pred'] = self.y_pred
        self.singleton_dataset['y_pred_proba'] = self.y_pred_proba
        return



import pandas as pd
pd.set_option('mode.chained_assignment', None)
import numpy as np
from datetime import timedelta
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def predict_stock_one_day_alpha(symbol: str):
    path_name = 'https://github.com/reckoning-machines/datasets/blob/main/betas_table_zip.zip?raw=true'
    df = pd.read_csv(path_name,compression = 'zip')

    
    df = df.loc[df['date']>'2019-01-01']
    df['target']=np.where(df['one_day_alpha']>0,1,0)
    df.columns
    featureset_columns = ['hyg_return', 'tlt_return', 'vb_return',
        'vtv_return', 'vug_return', 'rut_return', 'spx_return', 'DGS10_return',
        'DGS2_return', 'DTB3_return', 'DFF_return', 'T10Y2Y_return',
        'T5YIE_return', 'BAMLH0A0HYM2_return', 'DEXUSEU_return', 'KCFSI_return',
        'DRTSCILM_return', 'RSXFS_return', 'MARTSMPCSM44000USS_return',
        'H8B1058NCBCMG_return', 'DCOILWTICO_return', 'VXVCLS_return',
        'H8B1247NCBCMG_return', 'GASREGW_return',
        'CSUSHPINSA_return', 'UNEMPLOY_return']

    dataset = df.loc[df['symbol']==symbol]
    dataset['target'] = dataset['target'].shift(-1)
    prediction_record = dataset[-1:]
    dataset =  dataset.iloc[:-1]
    dataset[featureset_columns] = dataset[featureset_columns].diff(periods=1, axis=0)
    dataset.dropna(inplace=True)
    dataset.to_csv('dataset.csv',index=False)
    y = dataset['target']
    X = dataset[featureset_columns].astype(float)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 10)

    classifier = LogisticRegression(random_state = 10)
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)
    test_acc = accuracy_score(y_test, y_pred)

    y_pred = classifier.predict_proba(prediction_record[featureset_columns])
    weights = classifier.coef_

    full_X_pred = classifier.predict(X)
    full_X_pred_proba = classifier.predict_proba(X)
    X['y_pred'] = full_X_pred
    X['y_pred_proba'] = full_X_pred_proba[:,1]
    display(X.tail())

    clsWalkForward = WalkForward(classifier,dataset,'target',featureset_columns)
    clsWalkForward.fit()
    display(clsWalkForward.result_dataset.tail())
 
    weights = pd.DataFrame(zip(X_train.columns, np.transpose(weights[0])), columns=['features', 'coef']) 
    ticker_to_group = {
    'hyg_return': 'index',
    'tlt_return': 'index',
    'vb_return': 'index',
    'vtv_return':'index',
    'vug_return':'index',
    'rut_return':'index',
    'spx_return':'index',
    'DGS10_return':'bond market',
    'DGS2_return':'bond market',
    'DTB3_return':'bond market',
    'DFF_return':'bond market',
    'T10Y2Y_return':'bond market',
    'T5YIE_return':'bond market',
    'BAMLH0A0HYM2_return':'credit commodity and currency',
    'DEXUSEU_return':'credit commodity and currency',
    'KCFSI_return':'economic',
    'DRTSCILM_return':'economic',
    'RSXFS_return':'economic',
    'MARTSMPCSM44000USS_return':'economic',
    'H8B1058NCBCMG_return':'credit commodity and currency',
    'DCOILWTICO_return':'credit commodity and currency',
    'VXVCLS_return':'credit commodity and currency',
    'H8B1247NCBCMG_return':'credit commodity and currency',
    'GASREGW_return':'credit commodity and currency',
    'CSUSHPINSA_return':'credit commodity and currency',
    'UNEMPLOY_return':'economic'
    }
    ticker_to_english = {
    'hyg_return': 'High Yield',
    'tlt_return': 'Treasuries',
    'vb_return': 'Vanguard Small-Cap',
    'vtv_return':'Vanguard Value',
    'vug_return':'Vanguard Growth',
    'rut_return':'Russell 2000',
    'spx_return':'S&P 500',
    'DGS10_return':'GT10',
    'DGS2_return':'GT2',
    'DTB3_return':'GT3m',
    'DFF_return':'Fed Funds',
    'T10Y2Y_return':'GT10-GT2',
    'T5YIE_return':'5-Year Breakeven',
    'BAMLH0A0HYM2_return':'BofA US High Yield',
    'DEXUSEU_return':'USD Euro',
    'KCFSI_return':'KC Eco Index',
    'DRTSCILM_return':'Banks Tightening',
    'RSXFS_return':'Retail Sales',
    'MARTSMPCSM44000USS_return':'Retail Trade',
    'H8B1058NCBCMG_return':'Deposits',
    'DCOILWTICO_return':'WTI Crude',
    'VXVCLS_return':'S&P 500 3-Month Volatility',
    'H8B1247NCBCMG_return':'Consumer Loans',
    'GASREGW_return':'Gasoline',
    'CSUSHPINSA_return':'Case-Shiller',
    'UNEMPLOY_return':'Unemployment Rate'
    }

# Create a new column 'state' by mapping the 'city' column using the dictionary
    weights['group'] = weights['features'].map(ticker_to_group)
    weights['features'] = weights['features'].map(ticker_to_english)
#    print(weights)
    weights.to_csv('feature_importance.csv')

    trade_date = pd.to_datetime(prediction_record.iloc[0]['date']) + timedelta(days=1)
    
    return f"Model accuracy: {round(test_acc,2)}, ticker: {symbol},trade date:{trade_date},outperformance probability:{round(y_pred[:,0][0],2)}"
    
  

In [4]:
predict_stock_one_day_alpha('JPM US')


,hyg_return,tlt_return,vb_return,vtv_return,vug_return,rut_return,spx_return,DGS10_return,DGS2_return,DTB3_return,...,MARTSMPCSM44000USS_return,H8B1058NCBCMG_return,DCOILWTICO_return,VXVCLS_return,H8B1247NCBCMG_return,GASREGW_return,CSUSHPINSA_return,UNEMPLOY_return,y_pred,y_pred_proba
12588,0.002454,-0.005761,0.007850,0.003614,0.003902,0.006009,0.003246,0.004545,-0.000111,0.000000,...,0.0,0.0,0.002291,-0.026118,0.0,0.000000,0.0,0.0,0.0,0.476325
12589,0.000388,0.003784,0.000443,-0.006973,0.004391,0.003488,-0.000248,-0.004608,-0.012531,0.000000,...,0.0,0.0,-0.013325,0.024752,0.0,-0.006652,0.0,0.0,0.0,0.472747
12590,-0.001294,0.008112,-0.003765,0.004952,-0.001418,-0.005204,0.001585,-0.011282,0.002070,0.000000,...,0.0,0.0,0.003306,-0.019917,0.0,0.006652,0.0,0.0,0.0,0.480408
12591,-0.001424,-0.003957,-0.004267,-0.004539,-0.006119,-0.005966,-0.005208,0.011292,0.008299,0.000000,...,0.0,0.0,0.000000,0.037146,0.0,0.000000,0.0,0.0,0.0,0.475297
12592,-0.001302,-0.007635,-0.008490,-0.008757,-0.001875,-0.008062,-0.004675,0.004494,0.001989,0.001905,...,0.0,0.0,0.000000,-0.000649,0.0,0.000000,0.0,0.0,0.0,0.477046


,symbol,date,close,rsi,one_day_alpha,thirty_day_alpha,earnings_one_day_alpha,earnings_close_date,earnings_trade_date,fiscal_date_ending,...,hyg_return_coef,SP500_return_coef,DTB3_return_coef,VXVCLS_return_coef,T5YIE_return_coef,MARTSMPCSM44000USS_return_coef,alpha_target,target,y_pred,y_pred_proba
12587,JPM US,2024-05-16,202.47,71.372505,0.003590,0.067285,0.013267,2024-05-16,0,2024-06-30,...,1.210407,0.868954,-0.019182,-0.213158,0.196219,-0.000795,1,1.0,0.0,0.490633
12588,JPM US,2024-05-17,204.79,74.568947,0.010447,0.067293,-0.045985,2024-05-17,0,2024-06-30,...,1.209793,0.868727,-0.017514,-0.214252,0.196837,-0.000784,1,0.0,0.0,0.487401
12589,JPM US,2024-05-20,195.58,50.474690,-0.045764,0.067477,0.019354,2024-05-20,0,2024-06-30,...,1.211245,0.863538,-0.014926,-0.213505,0.193355,-0.000830,1,1.0,0.0,0.487050
12590,JPM US,2024-05-21,199.52,56.891764,0.017953,0.067024,-0.008257,2024-05-21,0,2024-06-30,...,1.206593,0.876321,-0.018832,-0.217581,0.192528,-0.000808,1,0.0,0.0,0.480914
12591,JPM US,2024-05-22,198.31,54.553950,-0.003696,0.067056,-0.004640,2024-05-22,0,2024-06-30,...,1.215190,0.875416,-0.017536,-0.220413,0.190906,-0.000803,1,0.0,0.0,0.474349


'Model accuracy: 0.52, ticker: JPM US,trade date:2024-05-25 00:00:00,outperformance probability:0.52'

In [5]:
clsWalkForward.result_dataset.tail()



NameError: name 'clsWalkForward' is not defined

In [4]:
import pandas as pd
data = pd.read_csv('feature_importance.csv')
data['all'] = 'all'
data['size']=abs(data['coef'])
data['coef'] = round(data['coef'].astype(float)*100,1)
data['size'] = round(data['size'].astype(float)*100,1)
data


,Unnamed: 0,features,coef,group,all,size
0,0,High Yield,11.8,index,all,11.8
1,1,Treasuries,-12.3,index,all,12.3
2,2,Vanguard Small-Cap,2.2,index,all,2.2
3,3,Vanguard Value,1.8,index,all,1.8
4,4,Vanguard Growth,17.0,index,all,17.0
5,5,Russell 2000,1.3,index,all,1.3
6,6,S&P 500,10.7,index,all,10.7
7,7,GT10,-2.3,bond market,all,2.3
8,8,GT2,19.4,bond market,all,19.4
9,9,GT3m,4.1,bond market,all,4.1


In [8]:
fig = px.treemap(data, path=[px.Constant('all'),'group','features','coef'], 
                 values='size')
my_colorway=("#636efa", "#e763fa", "#ab63fa")   

fig.update_layout(treemapcolorway=my_colorway,  
           width=1250, height=750, title_text="TreeMap JPM Feature Importance", title_x=0.5)
leafs = fig.data[0].labels[:len(data["coef"])]

leafs = [float(leaf) for leaf in leafs]

mycolors = ["#DC3714" if leafv < 0 else "#19BD1B" for leafv in leafs]
newlabels= [f"{fig.data[0].labels[k]}" if k < len(data["coef"]) else fig.data[0].labels[k] 
              for k in range(len(fig.data[0].labels))]

fig.update_traces(labels=newlabels)

fig.update_traces(marker_colors=mycolors, root_color="#c0c0c0")
fig.show()

NameError: name 'px' is not defined

In [ ]:
# symbol = "JPM US"
# modelbit.get_inference(
#   region="us-east-1",
#   workspace="reckoningmachines",
#   deployment="predict_stock_one_day_alpha",
#   data=symbol
# )

{'data': 'Model accuracy: 0.52, ticker: JPM US,trade date:2024-05-25 00:00:00,outperformance probability:0.52'}

In [10]:
dataset = pd.read_csv('dataset.csv')
dataset['target'] = dataset['target'].shift(-1)

In [12]:
if __name__ == "__main__":
    """
    Read sample dataset, do some target arranging / feature engineering
    """
    dataset = pd.read_csv('dataset.csv')
    symbol = "JPM US"
    featureset_columns = ['hyg_return', 'tlt_return', 'vb_return',
        'vtv_return', 'vug_return', 'rut_return', 'spx_return', 'DGS10_return',
        'DGS2_return', 'DTB3_return', 'DFF_return', 'T10Y2Y_return',
        'T5YIE_return', 'BAMLH0A0HYM2_return', 'DEXUSEU_return', 'KCFSI_return',
        'DRTSCILM_return', 'RSXFS_return', 'MARTSMPCSM44000USS_return',
        'H8B1058NCBCMG_return', 'DCOILWTICO_return', 'VXVCLS_return',
        'H8B1247NCBCMG_return', 'GASREGW_return',
        'CSUSHPINSA_return', 'UNEMPLOY_return']

    dataset = dataset.loc[dataset['symbol']==symbol]
    dataset['target'] = dataset['target'].shift(-1)
    prediction_record = dataset[-1:]
    dataset =  dataset.iloc[:-1]
    dataset[featureset_columns] = dataset[featureset_columns].diff(periods=1, axis=0)
    dataset.dropna(inplace=True)

    """
    Send dataset and simple binary classifier into WalkForward object and fit
    """
    classifier = LogisticRegression(random_state = 10)
    clsWalkForward = WalkForward(classifier,dataset,'target',featureset_columns)
    clsWalkForward.fit()
    print(clsWalkForward.result_dataset.tail())

